In [1]:
import pandas
pandas.options.mode.chained_assignment = None
import sys
import numpy as np
from globsML.utils.preprocessing import rescale_data, select_galaxies, create_data_dict, replace_NaN
from globsML.models.skmodels import StandardClassifier

data_path = '../../../data/ACS_sources_original.csv'
data = pandas.read_csv(data_path)
test_galaxies = set(data[data['cluster']=='Virgo']['galaxy'].unique())

In [2]:
df_train, dfs_test = select_galaxies(data, test_galaxies = test_galaxies)
df_train, dfs_test = replace_NaN(df_train, dfs_test)
df = create_data_dict(data, df_train, dfs_test)
df, _ = rescale_data(df)

input_dim = len(df['train']['inputs'][0])

Number of sources in training split after dropping rows with NaN as CI/m/color: 21232
535 sources have been dropped.
NaN values in testing data have been replaced with the correspnding median value observed in the training split


100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 3788.18it/s]

eccentricity will not be transformed. Skipped.
eccentricity_z will not be transformed. Skipped.


In [3]:
model = StandardClassifier(method='adaboost', data=df)

Loading data...
Fitting model...


In [4]:
model.val()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
model.stats_validation

,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,0.904153,0.072131,0.029372,0.954922,0.986766,283,313,22,1062


In [6]:
model.test()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dodo/git/globsML/globsML/utils/eval.py:146: RuntimeWarning: divide by zero encountered in double_scalars
  return auc(x,y)/(2*auc([0,x[-1]], [0,y[-1]])), x, y


In [7]:
model.stats_galaxies

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,VCC654,0.875000,0.086957,0.011905,0.936359,0.987444,42,48,4,384
1,VCC1283,0.953846,0.101449,0.002399,0.963377,0.996718,62,65,7,2983
2,VCC763,0.906504,0.011086,0.020921,0.978311,0.985241,446,492,5,731
3,VCC1030,0.918605,0.042424,0.015873,0.967038,0.987950,158,172,7,613
4,VCC543,0.846154,0.290323,0.014423,0.803549,0.985762,22,26,9,650
...,...,...,...,...,...,...,...,...,...,...
93,VCC1499,0.774194,0.200000,0.010830,0.803929,0.978252,24,31,6,585
94,VCC1087,0.850746,0.095238,0.013129,0.962566,0.993305,57,67,6,524
95,VCC1146,0.888889,0.027027,0.004515,0.974039,0.994914,72,81,2,524
96,VCC1895,0.769231,0.333333,0.005501,0.814318,0.994838,10,13,5,922


In [8]:
model.stats_galaxies.mean()

/tmp/ipykernel_1892/3415850949.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  model.stats_galaxies.mean()


TPR               0.842168
FDR               0.140742
FPR               0.015775
AUC(FDR,TPR)           inf
AUC(FPR,TPR)      0.983035
# found GCs     113.081633
# total GCs     126.479592
# fake GCs        7.142857
# sources       644.510204
dtype: float64

In [9]:
model.stats_all

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,ALL,0.89407,0.059413,0.013788,0.962974,0.989398,11082,12395,700,63162
